In [463]:
!pip install oracledb

In [464]:
import oracledb
import json
import random
from datetime import datetime

In [465]:
# ------------------- Conexão com o banco -------------------

In [466]:
def get_conn():
    with open("secret.txt", "r", encoding="utf-8") as f:
        creds = json.load(f)
    return oracledb.connect(
        user=creds["user"], 
        password=creds["password"], 
        dsn=creds["dsn"]
    )

In [467]:
# ------------------- Login e Cadastro -------------------

In [468]:
def register_user():
    conn = get_conn()
    cur = conn.cursor()

    print("\n=== CADASTRO ===")
    nome = input("Nome completo: ").strip()
    email = input("Email: ").strip()
    cpf = input("CPF (apenas números): ").strip() 
    
    senha = input("Senha: ").strip()

    try:
        cur.execute("""
            INSERT INTO users (name, email, cpf, password_hash, plan_id, role)
            VALUES (:1, :2, :3, :4, 1, 'user')
        """, (nome, email, cpf, senha)) # <-- Adicionar 'cpf' na tupla

        conn.commit()
        print("\n✅ Usuário cadastrado com sucesso (Plano Bronze).")

    except oracledb.IntegrityError as e:
        # É uma boa prática tratar exceções, caso o email ou CPF já existam
        error_obj, = e.args
        if error_obj.code == 1: # ORA-00001: unique constraint violated
             print("❌ Erro: Email ou CPF já cadastrado.")
        else:
             print(f"❌ Erro de integridade: {e}")
    
    except Exception as e:
        print(f"❌ Ocorreu um erro: {e}")

    finally:
        cur.close()
        conn.close()

In [469]:
def login_user():
    conn = get_conn()   
    cur = conn.cursor()

    print("\n=== LOGIN ===")
    email = input("Email: ").strip()
    senha = input("Senha: ").strip()

    cur.execute("""
        SELECT 
            u.id,
            u.name,
            u.role,
            u.plan_id,
            u.selected_challenge_id,
            p.name AS plan_name,
            p.description AS plan_description,
            p.has_ai_chat,
            p.has_specialist_chat
        FROM users u
        JOIN plans p ON u.plan_id = p.id
        WHERE u.email = :1 AND u.password_hash = :2
    """, (email, senha))

    row = cur.fetchone()

    if row is None:
        print("❌ Email ou senha incorretos.")
        cur.close()
        conn.close()
        return None

    user = {
        "id": row[0],
        "name": row[1],
        "role": row[2],
        "plan_id": row[3],
        "selected_challenge": row[4],
        "plan_name": row[5],
        "plan_description": row[6],
        "ai_chat": bool(row[7]),
        "specialist_chat": bool(row[8])
    }

    cur.close()
    conn.close()

    print(f"\n✅ Login bem-sucedido! Bem-vindo, {user['name']}!")
    print(f"Seu plano: {user['plan_name']}")
    return user

In [470]:
# MENU USUÁRIO

In [471]:
def user_menu(user):
    user_id = user["id"]
    selected_challenge = user["selected_challenge"]

    while True:
        print("\n=== MENU DO USUÁRIO ===")
        print(f"Usuário: {user['name']} | Plano: {user['plan_name']}")
        print("1 - Escolher desafio")
        print("2 - Jogar desafio selecionado")
        print("3 - Ver meu progresso")
        if user["ai_chat"]:
            print("4 - Chat IA (tira dúvidas)")
        if user["specialist_chat"]:
            print("5 - Chat com Especialista")
        print("0 - Logout")

        op = input("Escolha: ").strip()

        if op == "1":
            selected_challenge = choose_challenge(user_id)

        elif op == "2":
            if selected_challenge:
                play_selected_challenge(user_id, selected_challenge)
            else:
                print("⚠️ Nenhum desafio selecionado!")

        elif op == "3":
            view_progress(user_id)

        elif op == "4" and user["ai_chat"]:
            ia_chat_simulation()

        elif op == "5" and user["specialist_chat"]:
            specialist_chat_simulation()

        elif op == "0":
            print("🔒 Logout efetuado.")
            break

        else:
            print("⚠️ Opção inválida!")


In [472]:
# ESCOLHER DESAFIO

In [473]:
def choose_challenge(user_id):
    conn = get_conn()
    cur = conn.cursor()

    cur.execute("SELECT id, name FROM challenges ORDER BY id")
    desafios = cur.fetchall()

    if not desafios:
        print("⚠️ Nenhum desafio disponível.")
        cur.close()
        conn.close()
        return None

    print("\n=== DESAFIOS DISPONÍVEIS ===")
    for cid, nome in desafios:
        print(f"[{cid}] {nome}")

    escolha = input("Digite o ID do desafio: ").strip()

    if not escolha.isdigit() or int(escolha) not in [d[0] for d in desafios]:
        print("⚠️ ID inválido!")
        cur.close()
        conn.close()
        return None

    cid_int = int(escolha)

    cur.execute("""
        UPDATE users SET selected_challenge_id = :1 WHERE id = :2
    """, (cid_int, user_id))

    conn.commit()
    cur.close()
    conn.close()

    print(f"✅ Desafio {cid_int} selecionado!")
    return cid_int

In [474]:
# MINI JOGOS

In [475]:
def desafio_quiz():
    perguntas = [
        ("O que é uma soft skill?", "habilidade comportamental"),
        ("O que é importante para o trabalho em equipe?", "comunicação"),
        ("O que é empatia?", "entender o outro")
    ]
    acertos = 0
    for p, resp in perguntas:
        r = input(f"{p} ").lower().strip()
        if r == resp:
            acertos += 1
    score = int((acertos / len(perguntas)) * 100)
    print(f"Pontuação: {score}%")
    return score

In [476]:
def desafio_adivinhacao():
    numero = random.randint(1, 10)
    tentativas = 0
    print("🎯 Adivinhe o número entre 1 e 10!")
    while True:
        try:
            palpite = int(input("Seu palpite: "))
        except ValueError:
            print("Digite um número válido.")
            continue

        tentativas += 1

        if palpite == numero:
            print(f"Acertou em {tentativas} tentativas!")
            return max(100 - tentativas * 10, 0)

        elif palpite < numero:
            print("🔼 Maior!")
        else:
            print("🔽 Menor!")

In [477]:
def desafio_digitar():
    frase = "Aprender é evoluir constantemente"
    print(f"Digite a seguinte frase:\n> {frase}")
    entrada = input("Digite: ").strip()
    if entrada == frase:
        print("✅ Perfeito!")
        return 100
    else:
        print("❌ Erro na digitação!")
        return 50

In [478]:
# JOGAR DESAFIO SELECIONADO

In [479]:
def play_selected_challenge(user_id, challenge_id):
    print(f"\n🎯 Jogando desafio {challenge_id}...\n")

    if challenge_id == 1:
        score = desafio_quiz()
    elif challenge_id == 2:
        score = desafio_adivinhacao()
    elif challenge_id == 3:
        score = desafio_digitar()
    else:
        print("Desafio não implementado.")
        return

    update_progress(user_id, challenge_id, score)

In [480]:
# PROGRESSO

In [481]:
def update_progress(user_id, challenge_id, score):
    conn = get_conn()
    cur = conn.cursor()

    cur.execute("""
        MERGE INTO progress p
        USING dual
        ON (p.user_id = :1 AND p.challenge_id = :2)
        WHEN MATCHED THEN
            UPDATE SET p.score = :3, p.last_update = CURRENT_TIMESTAMP
        WHEN NOT MATCHED THEN
            INSERT (id, user_id, challenge_id, score)
            VALUES (progress_seq.nextval, :4, :5, :6)
    """, (user_id, challenge_id, score, user_id, challenge_id, score)) # <--- AQUI

    conn.commit()
    cur.close()
    conn.close()

    print("📈 Progresso atualizado!")

In [482]:
def view_progress(user_id):
    conn = get_conn()
    cur = conn.cursor()

    cur.execute("""
        SELECT c.name, p.score, p.last_update
        FROM progress p
        JOIN challenges c ON c.id = p.challenge_id
        WHERE p.user_id = :1
        ORDER BY c.id
    """, (user_id,))

    rows = cur.fetchall()
    cur.close()
    conn.close()

    if not rows:
        print("⚠️ Nenhum progresso encontrado.")
    else:
        print("\n=== SEU PROGRESSO ===")
        for nome, score, data in rows:
            print(f"- {nome}: {score}% ({data})")

In [483]:
#   CHATS SIMULADOS

In [484]:
def ia_chat_simulation():
    print("\n🤖 CHAT IA — digite 'sair' para encerrar.")
    while True:
        q = input("Você: ").strip().lower()
        if q == "sair":
            break
        print("IA: Sempre tente dividir problemas grandes em partes menores.")


In [485]:
def specialist_chat_simulation():
    print("\n🎓 CHAT ESPECIALISTA — digite 'sair' para encerrar.")
    while True:
        q = input("Você: ").strip().lower()
        if q == "sair":
            break
        print("Especialista: Trabalhe sua comunicação com prática diária e feedbacks reais.")

In [486]:
# MENU ADMIN

In [487]:
def admin_menu(user):
    while True:
        print("\n=== MENU ADMIN ===")
        print("1 - Criar novo desafio")
        print("2 - Ver desafios")
        print("3 - Ver progresso de todos")
        print("4 - Exportar progresso (JSON)")
        print("5 - Gerenciar usuários")
        print("0 - Sair")

        op = input("Escolha: ").strip()

        if op == "1":
            create_challenge()
        elif op == "2":
            list_challenges()
        elif op == "3":
            view_all_progress()
        elif op == "4":
            export_progress_json()
        elif op == "5":
            manage_users()
        elif op == "0":
            break
        else:
            print("Opção inválida.")

In [488]:
def create_challenge():
    conn = get_conn()
    cur = conn.cursor()

    nome = input("Nome do desafio: ").strip()
    desc = input("Descrição: ").strip()

    cur.execute("INSERT INTO challenges (name, description) VALUES (:1, :2)", (nome, desc))

    conn.commit()
    cur.close()
    conn.close()

    print("Desafio criado!")

In [489]:
def list_challenges():
    conn = get_conn()
    cur = conn.cursor()

    cur.execute("SELECT id, name FROM challenges ORDER BY id")
    rows = cur.fetchall()

    cur.close()
    conn.close()

    print("\n=== DESAFIOS ===")
    for r in rows:
        print(f"[{r[0]}] {r[1]}")

In [490]:
def view_all_progress():
    conn = get_conn()
    cur = conn.cursor()

    cur.execute("""
        SELECT u.name, c.name, p.score 
        FROM progress p
        JOIN users u ON u.id = p.user_id
        JOIN challenges c ON c.id = p.challenge_id
        ORDER BY u.name, c.id
    """)

    rows = cur.fetchall()
    cur.close()
    conn.close()

    print("\n=== PROGRESSO DE TODOS ===")
    for n, c, s in rows:
        print(f"{n} - {c}: {s}%")

In [491]:
def export_progress_json():
    conn = get_conn()
    cur = conn.cursor()

    cur.execute("""
        SELECT 
            u.name AS user_name, 
            c.name AS challenge_name, 
            p.score, 
            p.last_update
        FROM progress p
        JOIN users u ON u.id = p.user_id
        JOIN challenges c ON c.id = p.challenge_id
        ORDER BY u.name
    """)

    data = [dict(zip([d[0] for d in cur.description], row)) for row in cur.fetchall()]

    cur.close()
    conn.close()

    with open("export_progress.json", "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2, default=str)

    print("JSON exportado!")


In [492]:
def manage_users():
    conn = get_conn()
    cur = conn.cursor()

    cur.execute("""
        SELECT u.id, u.name, u.email, u.role, p.id, p.name
        FROM users u 
        JOIN plans p ON u.plan_id = p.id
        ORDER BY u.id
    """)

    users = cur.fetchall()

    print("\n=== USUÁRIOS ===")
    for u in users:
        print(f"{u[0]} - {u[1]} | {u[2]} | role: {u[3]} | plano: {u[5]}")

    uid = input("ID do usuário (enter para sair): ").strip()
    if not uid:
        return
    if not uid.isdigit():
        print("ID inválido.")
        return

    uid_int = int(uid)

    action = input("(p)romover admin, (c)mudar plano, (d)eletar, (v)oltar: ").strip().lower()

    if action == "p":
        cur.execute("UPDATE users SET role = 'admin' WHERE id = :1", (uid_int,))
        conn.commit()
        print("Promovido a admin!")

    elif action == "c":
        cur.execute("SELECT id, name FROM plans ORDER BY id")
        plans = cur.fetchall()
        print("Planos:")
        for pid, pname in plans:
            print(f"{pid} - {pname}")
        pid = int(input("Novo plano ID: "))
        cur.execute("UPDATE users SET plan_id = :1 WHERE id = :2", (pid, uid_int))
        conn.commit()
        print("Plano alterado!")

    elif action == "d":
        conf = input("Confirmar exclusão? (s/n): ").lower()
        if conf == "s":
            cur.execute("DELETE FROM users WHERE id = :1", (uid_int,))
            conn.commit()
            print("Usuário deletado.")

    cur.close()
    conn.close()

In [493]:
# MAIN

In [494]:
def main():
    while True:
        print("\n=== SkillUp - Aprendizado Gamificado ===")
        print("1 - Registrar")
        print("2 - Login")
        print("0 - Sair")

        op = input("Escolha: ").strip()

        if op == "1":
            register_user()

        elif op == "2":
            user = login_user()
            if user:
                if user["role"] == "admin":
                    admin_menu(user)
                else:
                    user_menu(user)

        elif op == "0":
            print("Até logo!")
            break

        else:
            print("Opção inválida!")

In [495]:
if __name__ == "__main__":
    main()


=== SkillUp - Aprendizado Gamificado ===
1 - Registrar
2 - Login
0 - Sair

=== CADASTRO ===
❌ Erro: Email ou CPF já cadastrado.

=== SkillUp - Aprendizado Gamificado ===
1 - Registrar
2 - Login
0 - Sair

=== LOGIN ===

✅ Login bem-sucedido! Bem-vindo, joao!
Seu plano: Bronze

=== MENU DO USUÁRIO ===
Usuário: joao | Plano: Bronze
1 - Escolher desafio
2 - Jogar desafio selecionado
3 - Ver meu progresso
0 - Logout

🎯 Jogando desafio 2...

🎯 Adivinhe o número entre 1 e 10!
🔽 Menor!
🔼 Maior!
Acertou em 3 tentativas!
📈 Progresso atualizado!

=== MENU DO USUÁRIO ===
Usuário: joao | Plano: Bronze
1 - Escolher desafio
2 - Jogar desafio selecionado
3 - Ver meu progresso
0 - Logout

=== SEU PROGRESSO ===
- Jogo de Adivinhação: 70% (2025-11-14 20:27:01.391735)

=== MENU DO USUÁRIO ===
Usuário: joao | Plano: Bronze
1 - Escolher desafio
2 - Jogar desafio selecionado
3 - Ver meu progresso
0 - Logout

=== DESAFIOS DISPONÍVEIS ===
[1] Quiz de Soft Skills
[2] Jogo de Adivinhação
[3] Desafio de Digitaçã